In [1]:
import os.path as op
import matplotlib
matplotlib.use('Agg')
import mne
mne.viz.set_3d_backend("pyvistaqt")
mne.viz.set_3d_options(antialias=False) 

import numpy as np

from mne.datasets import eegbci
from mne.datasets import fetch_fsaverage
# Download fsaverage files
fs_dir = fetch_fsaverage(verbose=True)
subjects_dir = op.dirname(fs_dir)

# The files live in:
subject = 'fsaverage'
trans = 'fsaverage'  # MNE has a built-in fsaverage transformation
src = op.join(fs_dir, 'bem', 'fsaverage-ico-5-src.fif')
bem = op.join(fs_dir, 'bem', 'fsaverage-5120-5120-5120-bem-sol.fif')

Using pyvistaqt 3d backend.

0 files missing from root.txt in /Users/schko/mne_data/MNE-fsaverage-data
0 files missing from bem.txt in /Users/schko/mne_data/MNE-fsaverage-data/fsaverage


In [2]:
import pandas as pd

import sys
 
# setting path
sys.path.append('..')

from os import listdir
from os.path import isfile, join
import pickle
from mna.utils.rnapp_data_format import read_all_lslpresets, return_metadata_from_name
eeg_channel='BioSemi'
eeg_montage='biosemi64'
data_dir = "../data/"
lsl_dir = "../mna/LSLPresets/"
onlyfiles = [f for f in listdir(data_dir) if isfile(join(data_dir, f)) and '.pkl' in f]
input_path = data_dir + onlyfiles[0] # pick a random file, idx 26 and 
print('input_path',input_path)
print(f"input_path {input_path}")
metadata_jsons = read_all_lslpresets(path_to_jsonfiles=lsl_dir)
with open(input_path, 'rb') as handle:
    rns_data = pickle.load(handle)
## Add metadata to data

for key in rns_data.keys():
    rns_data[key].append(return_metadata_from_name(key, metadata_jsons))



eeg_channel_names = mne.channels.make_standard_montage(eeg_montage).ch_names
df = pd.DataFrame(rns_data[eeg_channel][0], columns=rns_data[eeg_channel][1],
                  index=rns_data[eeg_channel][2]['ChannelNames']).T
starting_time_s = rns_data[eeg_channel][1][0]
freq = rns_data[eeg_channel][2]['NominalSamplingRate']
rna_channel_names = list(df.columns)
rna_channel_names[1:65] = eeg_channel_names
info = mne.create_info(ch_names=rna_channel_names, ch_types=['stim'] + ['eeg'] * 64 + ['ecg'] * 2 + ['misc'] * 22,
                       sfreq=freq)
info.set_montage(eeg_montage)
raw = mne.io.RawArray(df.T * 1e-6, info)
raw = raw.pick('eeg')
raw = raw.set_eeg_reference(ref_channels='average')  # set average reference

input_path ../data/09_24_2022_11_31_56-Exp_adadrive-Sbj_18-Ssn_3.dats.pkl
input_path ../data/09_24_2022_11_31_56-Exp_adadrive-Sbj_18-Ssn_3.dats.pkl
Creating RawArray with float64 data, n_channels=89, n_times=3088671
    Range : 0 ... 3088670 =      0.000 ...  1508.140 secs
Ready.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.


In [3]:
# Clean channel names to be able to use a standard 1005 montage
new_names = dict(
    (ch_name,
     ch_name.rstrip('.').upper().replace('Z', 'z').replace('FP', 'Fp'))
    for ch_name in raw.ch_names)
raw.rename_channels(new_names)

# Read and set the EEG electrode locations, which are already in fsaverage's
# space (MNI space) for standard_1020:
montage = mne.channels.make_standard_montage('biosemi64')

# Check that the locations of EEG electrodes is correct with respect to MRI
#mne.viz.plot_alignment(
#    raw.info, src=src, eeg=['original', 'projected'], trans=trans,
#    show_axes=True, mri_fiducials=True, dig='fiducials')
# raw.apply_proj()

In [4]:
fwd = mne.make_forward_solution(raw.info, trans=trans, src=src,
                                bem=bem, eeg=True, mindist=5.0, n_jobs=None)
print(fwd)

Source space          : /Users/schko/mne_data/MNE-fsaverage-data/fsaverage/bem/fsaverage-ico-5-src.fif
MRI -> head transform : /Users/schko/.conda/envs/mna/lib/python3.10/site-packages/mne/data/fsaverage/fsaverage-trans.fif
Measurement data      : instance of Info
Conductor model   : /Users/schko/mne_data/MNE-fsaverage-data/fsaverage/bem/fsaverage-5120-5120-5120-bem-sol.fif
Accurate field computations
Do computations in head coordinates
Free source orientations

Reading /Users/schko/mne_data/MNE-fsaverage-data/fsaverage/bem/fsaverage-ico-5-src.fif...
Read 2 source spaces a total of 20484 active source locations

Coordinate transformation: MRI (surface RAS) -> head
     0.999994  0.003552  0.000202      -1.76 mm
    -0.003558  0.998389  0.056626      31.09 mm
    -0.000001 -0.056626  0.998395      39.60 mm
     0.000000  0.000000  0.000000       1.00

Read  64 EEG channels from info
Head coordinate coil definitions created.
Source spaces are now in head coordinates.

Setting up the BEM 

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


    Found     0/ 7809 points outside using solid angles
    Total 10242/10242 points inside the surface
Interior check completed in 2219.1 ms
Checking surface interior status for 10242 points...
    Found  2241/10242 points inside  an interior sphere of radius   47.7 mm
    Found     0/10242 points outside an exterior sphere of radius   98.3 mm
    Found     0/ 8001 points outside using surface Qhull


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


    Found     0/ 8001 points outside using solid angles
    Total 10242/10242 points inside the surface
Interior check completed in 2196.7 ms

Setting up for EEG...


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.2s finished


Computing EEG at 20484 source locations (free orientations)...


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.



Finished.
<Forward | MEG channels: 0 | EEG channels: 64 | Source space: Surface with 20484 vertices | Source orientation: Free>


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.5s finished


In [5]:
mne.viz.plot_alignment(
    raw.info, src=src, eeg=['original', 'projected'], trans=trans,
    show_axes=True, mri_fiducials=True, dig='fiducials')

Reading /Users/schko/mne_data/MNE-fsaverage-data/fsaverage/bem/fsaverage-ico-5-src.fif...
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
Using outer_skin.surf for head surface.
Channel types::	eeg: 64
Projecting sensors to the head surface


In [6]:
inv = mne.minimum_norm.make_inverse_operator(raw.info, fwd, cov, loose=0.2) 

NameError: name 'cov' is not defined

In [7]:
cov = mne.compute_covariance(raw, method='auto')

AttributeError: 'RawArray' object has no attribute 'event_id'